In [3]:
import sys
import cv2
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QWidget, QVBoxLayout, QHBoxLayout, 
    QLineEdit, QPushButton, QTableWidget, QTableWidgetItem, QMessageBox, 
    QColorDialog, QFileDialog, QFontDialog, QSpinBox, QComboBox
)
from PyQt5.QtCore import Qt, QSettings
from PyQt5.QtGui import QColor, QFont
from screeninfo import get_monitors
import pandas as pd

class FoodManagementWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Food Management")
        self.setGeometry(150, 150, 400, 300)
        
        self.filename = "food_data.xlsx"

        label = QLabel("Food Management Window", self)
        label.setAlignment(Qt.AlignCenter)
        
        central_widget = QWidget(self)
        self.setCentralWidget(central_widget)
        
        layout = QVBoxLayout(central_widget)
        layout.addWidget(label)

        self.table = QTableWidget(0, 2)
        self.table.setHorizontalHeaderLabels(["Food Name", "Price"])
        layout.addWidget(self.table)

        form_layout = QHBoxLayout()
        self.food_name_input = QLineEdit()
        self.food_name_input.setPlaceholderText("Food Name")
        form_layout.addWidget(self.food_name_input)
        
        self.price_input = QLineEdit()
        self.price_input.setPlaceholderText("Price")
        form_layout.addWidget(self.price_input)
        
        add_button = QPushButton("Add")
        add_button.clicked.connect(self.add_food)
        form_layout.addWidget(add_button)
        
        layout.addLayout(form_layout)

        save_button = QPushButton("Save to Excel")
        save_button.clicked.connect(self.save_to_excel)
        layout.addWidget(save_button)
        
        load_button = QPushButton("Load from Excel")
        load_button.clicked.connect(self.load_from_excel)
        layout.addWidget(load_button)

        self.load_from_excel()

    def add_food(self):
        food_name = self.food_name_input.text()
        price = self.price_input.text()
        
        if not food_name or not price:
            QMessageBox.warning(self, "Input Error", "Both fields must be filled")
            return

        row_position = self.table.rowCount()
        self.table.insertRow(row_position)
        self.table.setItem(row_position, 0, QTableWidgetItem(food_name))
        self.table.setItem(row_position, 1, QTableWidgetItem(price))

        self.food_name_input.clear()
        self.price_input.clear()

    def save_to_excel(self):
        rows = []
        for row in range(self.table.rowCount()):
            food_name = self.table.item(row, 0).text()
            price = self.table.item(row, 1).text()
            rows.append({"Food Name": food_name, "Price": price})
        
        df = pd.DataFrame(rows)
        df.to_excel(self.filename, index=False)
        QMessageBox.information(self, "Saved", "Data has been saved to Excel")

    def load_from_excel(self):
        try:
            df = pd.read_excel(self.filename)
            self.table.setRowCount(0)
            for index, row in df.iterrows():
                row_position = self.table.rowCount()
                self.table.insertRow(row_position)
                self.table.setItem(row_position, 0, QTableWidgetItem(row["Food Name"]))
                self.table.setItem(row_position, 1, QTableWidgetItem(str(row["Price"])))
        except FileNotFoundError:
            pass

class SettingsWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Settings")
        self.setGeometry(200, 200, 800, 600)
        
        label = QLabel("Settings Window", self)
        label.setAlignment(Qt.AlignCenter)
        
        central_widget = QWidget(self)
        self.setCentralWidget(central_widget)
        
        grid_layout = QVBoxLayout(central_widget)
        grid_layout.addWidget(label)

        btn_color_button = QPushButton("Change Button Color")
        btn_color_button.clicked.connect(self.select_button_color)
        grid_layout.addWidget(btn_color_button)

        btn_image_button = QPushButton("Add Button Background Image")
        btn_image_button.clicked.connect(self.select_button_image)
        grid_layout.addWidget(btn_image_button)

        btn_font_button = QPushButton("Change Button Font")
        btn_font_button.clicked.connect(self.select_button_font)
        grid_layout.addWidget(btn_font_button)

        btn_text_size_button = QPushButton("Change Button Text Size")
        btn_text_size_button.clicked.connect(self.select_button_text_size)
        grid_layout.addWidget(btn_text_size_button)

        app_bg_color_button = QPushButton("Change App Background Color")
        app_bg_color_button.clicked.connect(self.select_app_bg_color)
        grid_layout.addWidget(app_bg_color_button)

        app_bg_image_button = QPushButton("Add App Background Image")
        app_bg_image_button.clicked.connect(self.select_app_bg_image)
        grid_layout.addWidget(app_bg_image_button)

        app_font_color_button = QPushButton("Change App Font Color")
        app_font_color_button.clicked.connect(self.select_app_font_color)
        grid_layout.addWidget(app_font_color_button)

        disp_bg_color_button = QPushButton("Change Display Background Color")
        disp_bg_color_button.clicked.connect(self.select_disp_bg_color)
        grid_layout.addWidget(disp_bg_color_button)

        disp_bg_image_button = QPushButton("Add Display Background Image")
        disp_bg_image_button.clicked.connect(self.select_disp_bg_image)
        grid_layout.addWidget(disp_bg_image_button)

        disp_font_color_button = QPushButton("Change Display Font Color")
        disp_font_color_button.clicked.connect(self.select_disp_font_color)
        grid_layout.addWidget(disp_font_color_button)

        # New settings
        font_size_label = QLabel("Display Font Size:")
        self.font_size_input = QSpinBox()
        self.font_size_input.setRange(8, 72)
        grid_layout.addWidget(font_size_label)
        grid_layout.addWidget(self.font_size_input)

        display_size_label = QLabel("Display Size:")
        self.display_size_input = QSpinBox()
        self.display_size_input.setRange(800, 3840)
        grid_layout.addWidget(display_size_label)
        grid_layout.addWidget(self.display_size_input)

        # Adding combo box for monitor or port selection
        display_type_label = QLabel("Select Display Type:")
        self.display_type_combo = QComboBox()
        grid_layout.addWidget(display_type_label)
        grid_layout.addWidget(self.display_type_combo)

        self.scan_displays()

        camera_ip_label = QLabel("Camera IP or Port:")
        self.camera_ip_input = QLineEdit()
        self.camera_ip_input.setPlaceholderText("Enter IP or Port")
        grid_layout.addWidget(camera_ip_label)
        grid_layout.addWidget(self.camera_ip_input)

        # Adding combo box for camera selection
        camera_label = QLabel("Select Camera:")
        self.camera_combo = QComboBox()
        grid_layout.addWidget(camera_label)
        grid_layout.addWidget(self.camera_combo)

        self.scan_cameras()

        apply_button = QPushButton("Apply Settings")
        apply_button.clicked.connect(self.apply_settings)
        grid_layout.addWidget(apply_button)

        close_button = QPushButton("Close Settings")
        close_button.clicked.connect(self.close_settings)
        grid_layout.addWidget(close_button)

        self.settings = QSettings("MyApp", "MyAppSettings")

    def scan_displays(self):
        monitors = get_monitors()
        for monitor in monitors:
            self.display_type_combo.addItem(f"Monitor: {monitor.name} ({monitor.width}x{monitor.height})")

    def scan_cameras(self):
        index = 0
        while True:
            cap = cv2.VideoCapture(index)
            if not cap.read()[0]:
                break
            self.camera_combo.addItem(f"Camera {index}")
            cap.release()
            index += 1

    def select_button_color(self):
        color = QColorDialog.getColor()
        if color.isValid():
            self.settings.setValue("btn_color", color.name())

    def select_button_image(self):
        file_dialog = QFileDialog()
        file_path, _ = file_dialog.getOpenFileName(self, "Select Image", "", "Image Files (*.png *.jpg *.bmp)")
        if file_path:
            self.settings.setValue("btn_image", file_path)

    def select_button_font(self):
        font, ok = QFontDialog.getFont()
        if ok:
            self.settings.setValue("btn_font", font.toString())

    def select_button_text_size(self):
        font, ok = QFontDialog.getFont()
        if ok:
            self.settings.setValue("btn_text_size", font.pointSize())

    def select_app_bg_color(self):
        color = QColorDialog.getColor()
        if color.isValid():
            self.settings.setValue("app_bg_color", color.name())

    def select_app_bg_image(self):
        file_dialog = QFileDialog()
        file_path, _ = file_dialog.getOpenFileName(self, "Select Image", "", "Image Files (*.png *.jpg *.bmp)")
        if file_path:
            self.settings.setValue("app_bg_image", file_path)

    def select_app_font_color(self):
        color = QColorDialog.getColor()
        if color.isValid():
            self.settings.setValue("app_font_color", color.name())

    def select_disp_bg_color(self):
        color = QColorDialog.getColor()
        if color.isValid():
            self.settings.setValue("disp_bg_color", color.name())

    def select_disp_bg_image(self):
        file_dialog = QFileDialog()
        file_path, _ = file_dialog.getOpenFileName(self, "Select Image", "", "Image Files (*.png *.jpg *.bmp)")
        if file_path:
            self.settings.setValue("disp_bg_image", file_path)

    def select_disp_font_color(self):
        color = QColorDialog.getColor()
        if color.isValid():
            self.settings.setValue("disp_font_color", color.name())

    def apply_settings(self):
        self.apply_screen_size()
        
        # Save new settings
        font_size = self.font_size_input.value()
        self.settings.setValue("display_font_size", font_size)

        display_size = self.display_size_input.value()
        self.settings.setValue("display_size", display_size)

        display_type = self.display_type_combo.currentText()
        self.settings.setValue("display_type", display_type)

        display_ip = self.camera_ip_input.text()
        self.settings.setValue("display_ip", display_ip)

        camera = self.camera_combo.currentText()
        self.settings.setValue("camera", camera)

    def apply_screen_size(self):
        settings = QSettings("MyApp", "MyAppSettings")
        screen_width = self.display_size_input.value()
        screen_height = self.height()
        settings.setValue("screen_width", screen_width)
        settings.setValue("screen_height", screen_height)
        self.resize(screen_width, screen_height)

    def close_settings(self):
        self.close()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("پنل مدیریت")
        self.setGeometry(100, 100, 1024, 865)

        button_width = 200
        button_height = 60

        self.food_management_button1 = QPushButton("مدیریت غذا", self)
        self.food_management_button1.setFixedSize(button_width, button_height)
        self.food_management_button1.clicked.connect(self.open_food_management)

        self.food_management_button2 = QPushButton("تنظیمات", self)
        self.food_management_button2.setFixedSize(button_width, button_height)
        self.food_management_button2.clicked.connect(self.open_settings)

        self.exit_button = QPushButton("خروج", self)
        self.exit_button.setFixedSize(button_width, button_height)
        self.exit_button.clicked.connect(self.close_application)

        central_widget = QWidget()
        self.setCentralWidget(central_widget)

        layout = QVBoxLayout(central_widget)
        layout.addStretch()
        layout.addWidget(self.food_management_button1, alignment=Qt.AlignCenter)
        layout.addWidget(self.food_management_button2, alignment=Qt.AlignCenter)
        layout.addWidget(self.exit_button, alignment=Qt.AlignCenter)
        layout.addStretch()

        self.load_button_settings()

    def open_food_management(self):
        self.food_management_window = FoodManagementWindow()
        self.food_management_window.show()

    def open_settings(self):
        self.settings_window = SettingsWindow()
        self.settings_window.show()

    def close_application(self):
        QApplication.instance().quit()

    def load_button_settings(self):
        settings = QSettings("MyApp", "MyAppSettings")

        btn_color = settings.value("btn_color", None)
        btn_image = settings.value("btn_image", None)
        btn_font = settings.value("btn_font", None)
        btn_text_size = settings.value("btn_text_size", None)

        app_bg_color = settings.value("app_bg_color", None)
        app_bg_image = settings.value("app_bg_image", None)
        app_font_color = settings.value("app_font_color", None)

        disp_bg_color = settings.value("disp_bg_color", None)
        disp_bg_image = settings.value("disp_bg_image", None)
        disp_font_color = settings.value("disp_font_color", None)

        display_font_size = settings.value("display_font_size", 12, int)
        display_size = settings.value("display_size", 1024, int)
        display_type = settings.value("display_type", "")
        display_ip = settings.value("display_ip", "")
        camera = settings.value("camera", "")

        buttons = [self.food_management_button1, self.food_management_button2, self.exit_button]

        for button in buttons:
            if btn_color:
                button.setStyleSheet(f"background-color: {btn_color};")
            if btn_image:
                button.setStyleSheet(f"background-image: url({btn_image});")
            if btn_font:
                font = QFont()
                font.fromString(btn_font)
                button.setFont(font)
            if btn_text_size:
                font = button.font()
                font.setPointSize(int(btn_text_size))
                button.setFont(font)

        # Handle other settings like app background color, image, and font color
        if app_bg_color:
            self.centralWidget().setStyleSheet(f"background-color: {app_bg_color};")
        if app_bg_image:
            self.centralWidget().setStyleSheet(f"background-image: url({app_bg_image});")
        if app_font_color:
            palette = self.centralWidget().palette()
            palette.setColor(self.centralWidget().foregroundRole(), QColor(app_font_color))
            self.centralWidget().setPalette(palette)

        # Handle display settings
        if disp_bg_color:
            # Example: set this background color to a QLabel as an example of display component
            pass
        if disp_bg_image:
            # Example: set this background image to a QLabel as an example of display component
            pass
        if disp_font_color:
            # Example: set this font color to a QLabel as an example of display component
            pass

        screen_width = settings.value("screen_width", 1024, int)
        screen_height = settings.value("screen_height", 865, int)
        self.resize(screen_width, screen_height)

        # Apply new settings
        font = QFont()
        font.setPointSize(display_font_size)
        self.centralWidget().setFont(font)
        self.resize(display_size, self.height())

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())


: 